<a href="https://colab.research.google.com/github/kunai-3txk/Compe_tonyobyo/blob/main/AMP_FeatureStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [2]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 49.8 MB/s eta 0:00:00


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling
import os
import seaborn as sns; sns.set()
import polars as pl

# Configuration

In [4]:
class config:
    MAX_PROTEINS = 20
    PROFILE_REPORT = True
    
class paths:
    SAMPLE_SUBMISSION = "/content/drive/MyDrive/AMP/example_test_files/sample_submission.csv"
    SUPPLEMENTAL_CLINICAL_DATA = "/content/drive/MyDrive/AMP/supplemental_clinical_data.csv"
    TRAIN_CLINICAL_DATA = "/content/drive/MyDrive/AMP/train_clinical_data.csv"
    TRAIN_PEPTIDES = "/content/drive/MyDrive/AMP/train_peptides.csv"
    TRAIN_PROTEINS = "/content/drive/MyDrive/AMP/train_proteins.csv"
    TEST_CLINICAL_DATA = "/content/drive/MyDrive/AMP/example_test_files/test.csv"
    TEST_PEPTIDES = "/content/drive/MyDrive/AMP/example_test_files/test_peptides.csv"
    TEST_PROTEINS = "/content/drive/MyDrive/AMP/example_test_files/test_proteins.csv"

# Function Definition

In [5]:
#統計量の作成
def make_statistic(df,group_col:str,agg_list:list):
    df_ret = df.groupby(group_col).agg([
        pl.mean(agg_list).suffix('_mean')
        ,pl.std(agg_list).suffix("_std")
        ,pl.median(agg_list).suffix("_med")
        #,pl.col(agg_list).max().suffix("_max") 
        #,pl.col(agg_list).min().suffix("_min") 
        #,pl.quantile(agg_list',0.25).suffix("_quan1")
        #,pl.quantile(agg_list',0.75).suffix("_quan3")
        ])
    return df_ret

# Load Data

In [66]:
train_clinical = pd.read_csv(paths.TRAIN_CLINICAL_DATA)
train_peptides = pd.read_csv(paths.TRAIN_PEPTIDES)
train_proteins = pd.read_csv(paths.TRAIN_PROTEINS)
supplemental_clinical = pd.read_csv(paths.SUPPLEMENTAL_CLINICAL_DATA)
#print('Correct label:Proteins shape: (232741, 5) Peptides shape: (981834, 6) Clinical shape: (2615, 8)')
print('Proteins shape:',train_proteins.shape)
print('Peptides shape:',train_peptides.shape)
print('Clinical shape:',train_clinical.shape)
print('supplemental_clinical shape:',supplemental_clinical.shape)

Proteins shape: (232741, 5)
Peptides shape: (981834, 6)
Clinical shape: (2615, 8)
supplemental_clinical shape: (2223, 8)


#Protain

In [24]:
train_proteins['UniProt'].value_counts()

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0
...,...,...,...,...,...
232736,58648_108,108,58648,Q9UBX5,27387.8
232737,58648_108,108,58648,Q9UHG2,369437.0
232738,58648_108,108,58648,Q9UKV8,105830.0
232739,58648_108,108,58648,Q9Y646,21257.6


In [42]:


train_proteins_ = pl.DataFrame(train_proteins).select([
    pl.mean('NPX').over('visit_id').alias('NPX_mean_by_id')
   ,pl.std('NPX').over('visit_id').alias('NPX_std_by_id')
   ,pl.median('NPX').over('visit_id').alias('NPX_median_by_id')
   ,pl.mean('NPX').over('UniProt').alias('NPX_mean_by_Uni')
   ,pl.std('NPX').over('UniProt').alias('NPX_std_by_Uni')
   ,pl.median('NPX').over('UniProt').alias('NPX_median_by_Uni')
])
train_proteins_.head()

,visit_id,visit_month,patient_id,UniProt,NPX,NPX_mean_by_id,NPX_std_by_id,NPX_median_by_id,NPX_mean_by_Uni,NPX_std_by_Uni,NPX_median_by_Uni
0,55_0,0,55,O00391,11254.3,3.180508e+06,2.672700e+07,123757.0,11641.264435,2817.003530,11546.40
1,55_0,0,55,O00533,732430.0,3.180508e+06,2.672700e+07,123757.0,511164.883903,235735.664694,483442.50
2,55_0,0,55,O00584,39585.8,3.180508e+06,2.672700e+07,123757.0,26505.529157,10705.152540,26529.70
3,55_0,0,55,O14498,41526.9,3.180508e+06,2.672700e+07,123757.0,27305.934884,8446.187506,26624.00
4,55_0,0,55,O14773,31238.0,3.180508e+06,2.672700e+07,123757.0,17688.295406,7166.325369,17399.60
...,...,...,...,...,...,...,...,...,...,...,...
232736,58648_108,108,58648,Q9UBX5,27387.8,2.742771e+06,2.011897e+07,126502.5,35529.240173,12677.150462,35137.65
232737,58648_108,108,58648,Q9UHG2,369437.0,2.742771e+06,2.011897e+07,126502.5,229862.538275,101793.312719,214051.00
232738,58648_108,108,58648,Q9UKV8,105830.0,2.742771e+06,2.011897e+07,126502.5,69924.939658,25943.573085,67571.70
232739,58648_108,108,58648,Q9Y646,21257.6,2.742771e+06,2.011897e+07,126502.5,19884.525447,5941.115219,19476.60


In [58]:
train_proteins_agg_ = train_proteins[['visit_id']].join(train_proteins_.to_pandas())
train_proteins_agg = train_proteins_agg_.groupby('visit_id').mean().reset_index()
train_proteins_agg.head()

,visit_id,NPX_mean_by_id,NPX_std_by_id,NPX_median_by_id,NPX_mean_by_Uni,NPX_std_by_Uni,NPX_median_by_Uni
0,10053_0,2.856580e+06,2.131630e+07,136462.0,3.409050e+06,916581.377953,3.328035e+06
1,10053_12,2.728871e+06,2.092162e+07,135413.0,3.312293e+06,892441.636201,3.232692e+06
2,10053_18,2.509967e+06,1.969453e+07,115075.5,2.730909e+06,737777.693851,2.664924e+06
3,10138_12,3.002583e+06,2.516170e+07,116174.0,2.600823e+06,701280.225874,2.538556e+06
4,10138_24,3.068891e+06,2.716806e+07,110548.0,2.596264e+06,701540.456219,2.533459e+06


In [60]:
train_proteins_toall = train_proteins_agg.copy()

#Peptide

In [9]:
train_peptides['UniProt'].value_counts()

P02787    51916
P02768    40900
P01024    35253
P02751    27370
P02649    23082
          ...  
Q6UX71      661
P01780      654
Q562R1      616
Q99832      606
Q99829      489
Name: UniProt, Length: 227, dtype: int64

In [51]:
train_peptides_ = pl.DataFrame(train_peptides).select([
    pl.mean('PeptideAbundance').over('visit_id').alias('Abu_mean_by_id')
   ,pl.std('PeptideAbundance').over('visit_id').alias('Abu_std_by_id')
   ,pl.median('PeptideAbundance').over('visit_id').alias('Abu_median_by_id')
   ,pl.mean('PeptideAbundance').over('Peptide').alias('Abu_mean_by_Pep')
   ,pl.std('PeptideAbundance').over('Peptide').alias('Abu_std_by_Pep')
   ,pl.median('PeptideAbundance').over('Peptide').alias('Abu_median_by_Pep')
])
train_peptides_.head()

Abu_mean_by_id,Abu_std_by_id,Abu_median_by_id,Abu_mean_by_Pep,Abu_std_by_Pep,Abu_median_by_Pep
f64,f64,f64,f64,f64,f64
748153.907014,4.2318e6,93134.8,11641.264435,2817.00353,11546.4
748153.907014,4.2318e6,93134.8,76367.164728,32366.189011,72213.95
748153.907014,4.2318e6,93134.8,145751.527996,64302.85968,138427.0
748153.907014,4.2318e6,93134.8,25062.222917,11221.845903,23616.2
748153.907014,4.2318e6,93134.8,24059.520484,11002.026192,22063.85


In [53]:
train_peptides_agg_ = train_peptides[['visit_id']].join(train_peptides_.to_pandas())
train_peptides_agg = train_peptides_agg_.groupby('visit_id').mean().reset_index()
train_peptides_agg.head()

,visit_id,Abu_mean_by_id,Abu_std_by_id,Abu_median_by_id,Abu_mean_by_Pep,Abu_std_by_Pep,Abu_median_by_Pep
0,10053_0,726248.393431,3.535602e+06,86319.90,780277.460651,251328.150119,754544.223273
1,10053_12,737183.385744,3.799654e+06,89289.30,808401.635177,261304.901353,781443.500883
2,10053_18,601466.784320,3.006568e+06,75650.85,635184.511432,206856.059806,613536.699014
3,10138_12,699099.199189,3.379573e+06,81345.40,615542.728515,201007.780017,594361.074532
4,10138_24,732120.888877,4.912602e+06,73176.35,629034.485041,205927.049943,607168.866203


In [59]:
train_peptides_toall = train_peptides_agg.copy()

#Clinical

In [ ]:
train_clinical['visit_month'].nunique()
train_clinical['visit_month'].unique()
train_clinical['visit_month'].value_counts()
#[  0,   3,   6,   9,  12,  18,  24,  30,  36,  42,  48,  54,  60, 72,  84,  96, 108] 17types
#⇒ Phase_1 初期 [0,3,6,9,12,18,24] Phase_2 中期 [30,36,42,48,54,60] Phase_3 後期[72,84,96,108]

In [ ]:
#pivot
train_clinical_pv=train_clinical.dropna(axis=0).pivot(index="patient_id",columns='visit_month',values='visit_id').reset_index()
train_clinical_pv.head()

#testデータ用の候補探し
test_patient_list = list(train_clinical_pv[
    ~train_clinical_pv[0].isna() & ~train_clinical_pv[6].isna() & ~train_clinical_pv[12].isna() & ~train_clinical_pv[24].isna()
]['patient_id'].values)
len(test_patient_list)

In [67]:
#欠損値処理
train_clinical.dropna(subset=['updrs_1'],inplace=True)
train_clinical.dropna(subset=['updrs_2'],inplace=True)

#upd23dの置き換え
train_clinical.replace({'On':1,'Off':2},inplace=True)

train_clinical.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2613 entries, 0 to 2614
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   visit_id                             2613 non-null   object 
 1   patient_id                           2613 non-null   int64  
 2   visit_month                          2613 non-null   int64  
 3   updrs_1                              2613 non-null   float64
 4   updrs_2                              2613 non-null   float64
 5   updrs_3                              2588 non-null   float64
 6   updrs_4                              1576 non-null   float64
 7   upd23b_clinical_state_on_medication  1288 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 183.7+ KB


In [ ]:
train_clinical_ = pl.DataFrame(train_clinical).groupby('visit_id').agg([
    pl.mean('updrs_1').suffix('_mean')
    ,pl.std('updrs_1').suffix("_std")
    ,pl.median('updrs_1').suffix("_med")
    ])

train_clinical_.head()

In [64]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html

train_clinical_rolling = train_clinical.copy()

for i in range(3):
  train_clinical_rolling['updrs_{0}_rolling'.format(i+1)] = train_clinical_rolling.groupby('patient_id')['updrs_{0}'.format(i+1)].rolling(3,min_periods=1).mean().reset_index(drop=True)
  train_clinical_rolling['updrs_{0}_ewm'.format(i+1)] = train_clinical_rolling.groupby('patient_id')['updrs_{0}'.format(i+1)].transform(lambda x: x.ewm(halflife=10).mean()).reset_index(drop=True)
train_clinical_rolling.head()


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,updrs_1_rolling,updrs_1_ewm,updrs_2_rolling,updrs_2_ewm,updrs_3_rolling,updrs_3_ewm
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,10.000000,10.000000,6.000000,6.000000,15.000000,15.000000
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN,10.000000,10.000000,6.500000,6.517322,20.000000,20.173217
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN,9.333333,9.286627,7.666667,7.759546,24.666667,25.105042
3,55_9,55,9,8.0,9.0,30.0,0.0,On,8.666667,8.930796,8.666667,8.102607,29.666667,26.458798
4,55_12,55,12,10.0,10.0,41.0,0.0,On,8.666667,9.175259,9.666667,8.536426,35.000000,29.783493


# supplemental_clinical

In [23]:
supplemental_clinical

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,35_0,35,0,5.0,3.0,16.0,0.0,NaN
1,35_36,35,36,6.0,4.0,20.0,0.0,NaN
2,75_0,75,0,4.0,6.0,26.0,0.0,NaN
3,75_36,75,36,1.0,8.0,38.0,0.0,On
4,155_0,155,0,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...
2218,65382_0,65382,0,NaN,NaN,0.0,NaN,NaN
2219,65405_0,65405,0,5.0,16.0,31.0,0.0,NaN
2220,65405_5,65405,5,NaN,NaN,57.0,NaN,NaN
2221,65530_0,65530,0,10.0,6.0,24.0,0.0,NaN


#Train All

In [68]:
def make_train_all(df_cli,df_pro,df_pep):
  df_propep = df_pro.merge(df_pep,on='visit_id',how='left')
  df_all = df_cli.merge(df_propep,on='visit_id',how='left')
  return df_all


train_all = make_train_all(train_clinical,train_proteins_toall,train_peptides_toall)
print('shape',train_all.shape)
train_all.head()



shape (2613, 20)


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,NPX_mean_by_id,NPX_std_by_id,NPX_median_by_id,NPX_mean_by_Uni,NPX_std_by_Uni,NPX_median_by_Uni,Abu_mean_by_id,Abu_std_by_id,Abu_median_by_id,Abu_mean_by_Pep,Abu_std_by_Pep,Abu_median_by_Pep
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,3.180508e+06,2.672700e+07,123757.0,2.596938e+06,702016.931653,2.534106e+06,748153.907014,4.231806e+06,93134.8,619334.666897,202617.423261,597764.341992
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN,2.942039e+06,2.391469e+07,129048.0,2.608845e+06,705203.512224,2.545728e+06,685218.599872,3.491859e+06,84814.9,620241.001591,203152.040437,598682.013953
3,55_9,55,9,8.0,9.0,30.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55_12,55,12,10.0,10.0,41.0,0.0,1.0,3.145608e+06,2.612674e+07,124498.0,2.562630e+06,692742.468271,2.500628e+06,740535.423563,3.882536e+06,91575.2,612046.308868,200234.524653,590729.986479


In [69]:
train_all.to_csv('/content/drive/MyDrive/AMP/train_all.csv')